## Summary MLO component proteins in human and mouse from DrLLPS and PhaSepDB

In [1]:
import pandas as pd
import numpy as np

import bs4 as bs
import urllib
import requests

In [2]:
def humanToMouse_id(uniprot_id_human):    
    """find mouse uniprot id for each protein according to its human uniprot id"""
    try:
        url = "https://www.uniprot.org/uniprot/{0}" . format(uniprot_id_human)
        with requests.Session() as s:
            sauce = s.get(url, timeout = 10)
            soup = bs.BeautifulSoup(sauce.content,'html.parser')
            section = soup.find("section", id="page-header")
            divs = section.find_all("div")
            h2 = divs[1].find("h2")
            title = h2.find("span")
            uniprot_genename_human = title.get_text().lstrip("(").rstrip(")")
        
        print(uniprot_genename_human)
        
        url_mouse = ""
        url_mouse = "https://www.uniprot.org/uniprot/?query={0}&sort=score" . format(uniprot_genename_human.replace("HUMAN", "MOUSE"))

        with requests.Session() as s:
            sauce = s.get(url_mouse, timeout = 10)
            uniprot_id_mouse = str(sauce.url).split("/")[-1]
            
        if "?" not in uniprot_id_mouse:
             return uniprot_id_mouse
        else:
            return None
    except:
        return None

### (1) DrLLPS

In [3]:
df_DrLLPS = pd.read_csv("/mnt/Storage/home/yuzhaowei/projects/Factor.Harbor/data/public/LLPS/DrLLPS/LLPS.txt", header = 0, sep = "\t")

# human
df_DrLLPS_human = df_DrLLPS.loc[(
            (df_DrLLPS["Species"] == "Homo sapiens") & 
            ((df_DrLLPS["LLPS Type"] == "Scaffold") | (df_DrLLPS["LLPS Type"] == "Client") | (df_DrLLPS["LLPS Type"] == "Regulator"))
            ), :]
DrLLPS_condensate_human = np.unique(np.concatenate([np.array(row.split(", ")) for row in df_DrLLPS_human.loc[:,"Condensate"].values]))
DrLLPS_condensate_protein_human = []
for condensate in np.setdiff1d(DrLLPS_condensate_human, ["Droplet", "Others"]):
    condensate_protein = np.unique(df_DrLLPS_human.loc[df_DrLLPS_human["Condensate"].str.contains(condensate),"UniProt ID"].dropna().values)
    DrLLPS_condensate_protein_human.append([condensate, condensate_protein])
    
# mouse
df_DrLLPS_mouse = df_DrLLPS.loc[(
            (df_DrLLPS["Species"] == "Mus musculus") & 
            ((df_DrLLPS["LLPS Type"] == "Scaffold") | (df_DrLLPS["LLPS Type"] == "Client") | (df_DrLLPS["LLPS Type"] == "Regulator"))
            ), :]
DrLLPS_condensate_mouse = np.unique(np.concatenate([np.array(row.split(", ")) for row in df_DrLLPS_mouse.loc[:,"Condensate"].values]))
DrLLPS_condensate_protein_mouse = []
for condensate in np.setdiff1d(DrLLPS_condensate_mouse, ["Droplet", "Others"]):
    condensate_protein = np.unique(df_DrLLPS_mouse.loc[df_DrLLPS_mouse["Condensate"].str.contains(condensate),"UniProt ID"].dropna().values)
    DrLLPS_condensate_protein_mouse.append([condensate, condensate_protein])


### (2) PhaSepDB (v1)

In [4]:
df_PhaSepDB_reviewed = pd.read_csv("/mnt/Storage/home/yuzhaowei/projects/Factor.Harbor/data/public/LLPS/PhaSepDB/Reviewed_Data_V1.3.txt", header = 1, sep = "\t")
df_PhaSepDB_reviewed_human = df_PhaSepDB_reviewed.loc[((df_PhaSepDB_reviewed["Organism"] == "Homo sapiens ") | (df_PhaSepDB_reviewed["Organism"] == "Homo sapiens")),:]
df_PhaSepDB_reviewed_human_MLO = df_PhaSepDB_reviewed_human.loc[~df_PhaSepDB_reviewed_human["Body"].isnull(), ["UniprotEntry", "GeneSymbol", "Organism", "Body"]]
df_PhaSepDB_reviewed_mouse = df_PhaSepDB_reviewed.loc[df_PhaSepDB_reviewed["Organism"] == "Mus musculus",:]
df_PhaSepDB_reviewed_mouse_MLO = df_PhaSepDB_reviewed_mouse.loc[~df_PhaSepDB_reviewed_mouse["Body"].isnull(), ["UniprotEntry", "GeneSymbol", "Organism", "Body"]]

df_PhaSepDB_UniProtReviewed = pd.read_csv("/mnt/Storage/home/yuzhaowei/projects/Factor.Harbor/data/public/LLPS/PhaSepDB/UniProtReviewed_Data_V1.3.txt", header = 1, sep = "\t")
df_PhaSepDB_UniProtReviewed_human = df_PhaSepDB_UniProtReviewed.loc[((df_PhaSepDB_UniProtReviewed["Organism"] == "Homo sapiens ") | (df_PhaSepDB_UniProtReviewed["Organism"] == "Homo sapiens")),:]
df_PhaSepDB_UniProtReviewed_human_MLO = df_PhaSepDB_UniProtReviewed_human.loc[~df_PhaSepDB_UniProtReviewed_human["Body"].isnull(), ["UniprotEntry", "GeneSymbol", "Organism", "Body"]]
df_PhaSepDB_UniProtReviewed_mouse = df_PhaSepDB_UniProtReviewed.loc[df_PhaSepDB_UniProtReviewed["Organism"] == "Mus musculus",:]
df_PhaSepDB_UniProtReviewed_mouse_MLO = df_PhaSepDB_UniProtReviewed_mouse.loc[~df_PhaSepDB_UniProtReviewed_mouse["Body"].isnull(), ["UniprotEntry", "GeneSymbol", "Organism", "Body"]]

df_PhaSepDB_high_throughput = pd.read_csv("/mnt/Storage/home/yuzhaowei/projects/Factor.Harbor/data/public/LLPS/PhaSepDB/High_throughput_Data_V1.3.txt", header = 1, sep = "\t")
df_PhaSepDB_high_throughput_human = df_PhaSepDB_high_throughput.loc[((df_PhaSepDB_high_throughput["Organism"] == "Homo sapiens ") | (df_PhaSepDB_high_throughput["Organism"] == "Homo sapiens")), :]
df_PhaSepDB_high_throughput_human_MLO = df_PhaSepDB_high_throughput_human.loc[~df_PhaSepDB_high_throughput_human["Body"].isnull(), ["UniprotEntry", "GeneSymbol", "Organism", "Body"]]
df_PhaSepDB_high_throughput_mouse = df_PhaSepDB_high_throughput.loc[df_PhaSepDB_high_throughput["Organism"] == "Mus musculus", :]
df_PhaSepDB_high_throughput_mouse_MLO = df_PhaSepDB_high_throughput_mouse.loc[~df_PhaSepDB_high_throughput_mouse["Body"].isnull(), ["UniprotEntry", "GeneSymbol", "Organism", "Body"]]

df_PhaSepDB_human_MLO = pd.concat([df_PhaSepDB_reviewed_human_MLO, df_PhaSepDB_UniProtReviewed_human_MLO, df_PhaSepDB_high_throughput_human_MLO], axis = 0)
df_PhaSepDB_mouse_MLO = pd.concat([df_PhaSepDB_reviewed_mouse_MLO, df_PhaSepDB_UniProtReviewed_mouse_MLO, df_PhaSepDB_high_throughput_mouse_MLO], axis = 0)

PhaSepDB_condensate_protein_human = []
PhaSepDB_condensate_human = np.unique(df_PhaSepDB_human_MLO.loc[:, "Body"].values)
for condensate in PhaSepDB_condensate_human:
    condensate_protein = np.unique(df_PhaSepDB_human_MLO.loc[df_PhaSepDB_human_MLO["Body"] == condensate, "UniprotEntry"].dropna().values)
    PhaSepDB_condensate_protein_human.append([condensate, condensate_protein])
    
PhaSepDB_condensate_protein_mouse = []
PhaSepDB_condensate_mouse = np.unique(df_PhaSepDB_mouse_MLO.loc[:, "Body"].values)
for condensate in PhaSepDB_condensate_mouse:
    condensate_protein = np.unique(df_PhaSepDB_mouse_MLO.loc[df_PhaSepDB_mouse_MLO["Body"] == condensate, "UniprotEntry"].dropna().values)
    PhaSepDB_condensate_protein_mouse.append([condensate, condensate_protein])

### (3) PhaSepDB (v2)

In [5]:
df_PhaSepDB_v2_MLO = pd.read_excel("~/projects/Factor.Harbor/data/public/LLPS/PhaSepDB_v2/phasepdbv2_mlo.xlsx")
df_PhaSepDB_v2_human_MLO = df_PhaSepDB_v2_MLO.loc[df_PhaSepDB_v2_MLO["organism"] == "Homo sapiens", ["entry", "name", "organism", "MLO"]]
df_PhaSepDB_v2_human_MLO.columns = ["UniprotEntry", "GeneSymbol", "Organism", "Body"]
df_PhaSepDB_v2_mouse_MLO = df_PhaSepDB_v2_MLO.loc[df_PhaSepDB_v2_MLO["organism"] == "Mus musculus", ["entry", "name", "organism", "MLO"]]
df_PhaSepDB_v2_mouse_MLO.columns = ["UniprotEntry", "GeneSymbol", "Organism", "Body"]

PhaSepDB_v2_condensate_protein_human = []
PhaSepDB_v2_condensate_human = np.unique(df_PhaSepDB_v2_human_MLO.loc[:, "Body"].values)
for condensate in PhaSepDB_v2_condensate_human:
    condensate_protein = np.unique(df_PhaSepDB_v2_human_MLO.loc[df_PhaSepDB_v2_human_MLO["Body"] == condensate, "UniprotEntry"].dropna().values)
    PhaSepDB_v2_condensate_protein_human.append([condensate, condensate_protein])

PhaSepDB_v2_condensate_protein_mouse = []
PhaSepDB_v2_condensate_mouse = np.unique(df_PhaSepDB_v2_mouse_MLO.loc[:, "Body"].values)
for condensate in PhaSepDB_v2_condensate_mouse:
    condensate_protein = np.unique(df_PhaSepDB_v2_mouse_MLO.loc[df_PhaSepDB_v2_mouse_MLO["Body"] == condensate, "UniprotEntry"].dropna().values)
    PhaSepDB_v2_condensate_protein_mouse.append([condensate, condensate_protein])
    

### (4) Merge all MLOs

In [6]:
# show all MLOs
DrLLPS_MLOs_human = [condensate for condensate, condensate_protein in DrLLPS_condensate_protein_human]
PhaSepDB_MLOs_human = [condensate for condensate, condensate_protein in PhaSepDB_condensate_protein_human]
PhaSepDB_v2_MLOs_human = [condensate for condensate, condensate_protein in PhaSepDB_v2_condensate_protein_human]

DrLLPS_MLOs_mouse = [condensate for condensate, condensate_protein in DrLLPS_condensate_protein_mouse]
PhaSepDB_MLOs_mouse = [condensate for condensate, condensate_protein in PhaSepDB_condensate_protein_mouse]
PhaSepDB_v2_MLOs_mouse = [condensate for condensate, condensate_protein in PhaSepDB_v2_condensate_protein_mouse]

In [7]:
print(DrLLPS_MLOs_human)
print(PhaSepDB_MLOs_human)
print(PhaSepDB_v2_MLOs_human)

['Balbiani body', 'Cajal body', 'Centrosome/Spindle pole body', 'Chromatoid body', 'Cleavage body', 'DDX1 body', 'DNA damage foci', 'Gemini of cajal body', 'Histone locus body', 'Microtubule', 'Mitochondrial RNA granule', 'Neuronal granule', 'Nuage', 'Nuclear pore complex', 'Nuclear speckle', 'Nuclear stress body', 'Nucleolus', 'OPT domain', 'P-body', 'PML nuclear body', 'Paraspeckle', 'PcG body', 'Pericentriolar matrix', 'Perinucleolar compartment', 'Postsynaptic density', 'Receptor cluster', 'Sam68 nuclear body', 'Spindle apparatus', 'Stress granule', 'U body']
['Cajal body', 'Centrosome', 'Cytoplasmic granule', 'Cytoplasmic ribonucleoprotein granule', 'Gem', 'Heterochromatin', 'Histone Locus body', 'Histone locus body', 'Inflammasome', 'Membrane clusters', 'Neuronal inclusions', 'Nuclear Compartments', 'Nuclear body', 'Nuclear pore complex', 'Nuclear speckles', 'Nucleolus', 'Nucleus speckles', 'P granule', 'P-body', 'P-body, GW body', 'P-body, Stress granule', 'PML body', 'Paraspeck

In [8]:
print(DrLLPS_MLOs_mouse)
print(PhaSepDB_MLOs_mouse)
print(PhaSepDB_v2_MLOs_mouse)

['Cajal body', 'Centrosome/Spindle pole body', 'Chromatoid body', 'Histone locus body', 'Neuronal granule', 'Nuage', 'Nuclear speckle', 'Nucleolus', 'P-body', 'PML nuclear body', 'Paraspeckle', 'Postsynaptic density', 'Receptor cluster', 'Sam68 nuclear body', 'Spindle apparatus', 'Stress granule']
['Paraspeckle', 'Pre and postsynaptic densities', 'Signaling puncta']
['Nuclear speckle', 'P-body', 'PML nuclear body', 'Paraspeckle', 'Rosenthal fiber', 'Stress granule', 'Tau condensate']


In [9]:
dict_MLOs = {"Balbiani body" : ["Balbiani body"],
             "Cajal body" : ["Cajal body"],
             "Centrosome" : ["Centrosome/Spindle pole body", "Centrosome"],
             "Chromatoid body" : ["Chromatoid body"],
             "Cleavage body" : ["Cleavage body"],
             "Cytoplasmic granule" : ["Cytoplasmic granule"],
             "Cytoplasmic ribonucleoprotein granule": ["Cytoplasmic ribonucleoprotein granule"],
             "DDX1 body" : ["DDX1 body"],
             "DNA damage foci" : ["DNA damage foci"],
             "Gem" : ["Gem"],
             "Gemini of cajal body" : ["Gemini of cajal body"],
             "Heterochromatin" : ["Heterochromatin"],
             "Histone locus body": ["Histone locus body", "Histone Locus body"],
             "Inflammasome" : ["Inflammasome"],
             "Membrane clusters" : ["Membrane clusters"],
             "miRISC" : ["miRISC"],
             "Microtubule" : ["Microtubule"],
            "Mitochondrial RNA granule" : ["Mitochondrial RNA granule"],
            "Neuronal granule" : ["Neuronal granule"], 
            "Neuronal inclusions": ["Neuronal inclusions"], 
            "Nuage" : ["Nuage"],
            "Nuclear Compartments" : ["Nuclear Compartments"],
            "Nuclear body" : ["Nuclear body"],
            "Nuclear pore complex" : ["Nuclear pore complex"],
            "Nuclear speckle" : ["Nuclear speckle", "Nuclear speckles", "Nuclues speckles"],
            "Nuclear stress body" : ["Nuclear stress body"],
            "Nucleolus" : ["Nucleolus"],
            "OPT domain" : ["OPT domain"],
            "P granule" : ["P granule"],
            "P-body" : ["P-body"],
            "PML nuclear body" : ["PML nuclear body", "PML body"],
            "Paraspeckle" : ["Paraspeckle"],
            "PcG body" : ["PcG body", "Polycomb bodies", "Polycomb body"],
            "Pericentriolar matrix" : ["Pericentriolar matrix"],
            "Perinucleolar compartment" : ["Perinucleolar compartment"],
            "Postsynaptic density" : ["Postsynaptic density", "Pre and postsynaptic densities", "Pre and postsynaptic density"],
             "Receptor cluster" : ["Receptor cluster"],
            "RNP granules" : ["RNP granules", "IMP1 ribonucleoprotein granule"],
            "Rosenthal fiber" : ["Rosenthal fiber"],
            "SMN graule": ["SMN granule", "SMN complex"],
            "Sam68 nuclear body" : ["Sam68 nuclear body", "Sam68 nuclear bodies"],
            "Signaling puncta" : ["Signaling puncta"],
            "Spindle apparatus" : ["Spindle apparatus", "Spindle pole"],
            "Splicesome" : ["Splicesome"],
            "Stress granule" : ["Stress granule", "Sress granule"],
            "TAU protein" : ["TAU protein"],
            "Transcription Factories" : ["Transcription Factories"],
            "U body" : ["U body"],
            "Z granule" : ["Z granule"]
}

In [10]:
print(len(dict_MLOs))

49


#### stat

In [19]:
# human MLOs
merged_condensate_protein_human = []
for MLO in dict_MLOs:
    condensate_proteins = []
    for condensate, condensate_protein in DrLLPS_condensate_protein_human:
        if condensate in dict_MLOs[MLO]:
            condensate_proteins.append(condensate_protein)
            break
    for condensate, condensate_protein in PhaSepDB_condensate_protein_human:
        if condensate in dict_MLOs[MLO]:
            condensate_proteins.append(condensate_protein)
            break
    for condensate, condensate_protein in PhaSepDB_v2_condensate_protein_human:
        if condensate in dict_MLOs[MLO]:
            condensate_proteins.append(condensate_protein)
            break
    
    if len(condensate_proteins) > 0:
        merged_condensate_proteins = np.unique(np.concatenate(condensate_proteins))
        merged_condensate_protein_human.append([MLO, merged_condensate_proteins])

# mouse MLOs
merged_condensate_protein_mouse = []
for MLO in dict_MLOs:
    condensate_proteins = []
    for condensate, condensate_protein in DrLLPS_condensate_protein_mouse:
        if condensate in dict_MLOs[MLO]:
            condensate_proteins.append(condensate_protein)
            break
    for condensate, condensate_protein in PhaSepDB_condensate_protein_mouse:
        if condensate in dict_MLOs[MLO]:
            condensate_proteins.append(condensate_protein)
            break
    for condensate, condensate_protein in PhaSepDB_v2_condensate_protein_mouse:
        if condensate in dict_MLOs[MLO]:
            condensate_proteins.append(condensate_protein)
            break
    
    if len(condensate_proteins) > 0:
        merged_condensate_proteins = np.unique(np.concatenate(condensate_proteins))
        merged_condensate_protein_mouse.append([MLO, merged_condensate_proteins])

In [22]:
len(merged_condensate_protein_mouse)

18

In [11]:
with open("MLO_human_merged.txt", "w") as outf:
    outf.write("MLO\tUniprot_IDs\n")
    for condensate, condensate_protein in merged_condensate_protein_human:
        outf.write("{0}\t{1}\n" . format(condensate, "," . join(condensate_protein)))
outf.close()

In [12]:
with open("MLO_mouse_merged.txt", "w") as outf:
    outf.write("MLO\tUniprot_IDs\n")
    for condensate, condensate_protein in merged_condensate_protein_mouse:
        outf.write("{0}\t{1}\n" . format(condensate, "," . join(condensate_protein)))
outf.close()

### (5). Mouse extended (plus human MLO proteins)

In [ ]:
with open("MLO_mouseExtended_merged.txt", "w") as outf:
    outf.write("MLO\tUniprot_IDs\n")
    for condensate, condensate_protein in merged_condensate_protein_human:
        condensate_protein_humanToMouse = []
        for uniprot_id_human in condensate_protein:
            uniprot_id_mouse = humanToMouse_id(uniprot_id_human)
            if uniprot_id_mouse != None:
                condensate_protein_humanToMouse.append(uniprot_id_mouse)
        outf.write("{0}\t{1}\n" . format(condensate + "_HUMAN", "," . join(np.unique(condensate_protein_humanToMouse))))
    
    for condensate, condensate_protein in merged_condensate_protein_mouse:
        outf.write("{0}\t{1}\n" . format(condensate, "," . join(condensate_protein)))
outf.close()

RBPS2_HUMAN
DAZP1_HUMAN
BMR1B_HUMAN
NOP56_HUMAN
PEX12_HUMAN
TERT_HUMAN
PAHX_HUMAN
XPO1_HUMAN
DKC1_HUMAN
PLXA2_HUMAN
PHF2_HUMAN
PEX14_HUMAN
WBP4_HUMAN
CIB2_HUMAN
PRP6_HUMAN
MED26_HUMAN
PARN_HUMAN
RPA1_HUMAN
CDK1_HUMAN
NPM_HUMAN
PGK2_HUMAN
FES_HUMAN
RU2B_HUMAN
ROS1_HUMAN
RU2A_HUMAN
PA2GA_HUMAN
UBF1_HUMAN
VGFR1_HUMAN
NUCL_HUMAN
ERBB3_HUMAN
FBRL_HUMAN
KAPCB_HUMAN
APEX1_HUMAN
PDE4A_HUMAN
LTK_HUMAN
GLPK_HUMAN
T2FA_HUMAN
FLT3_HUMAN
COIL_HUMAN
NAA10_HUMAN
ABL2_HUMAN
TCPG_HUMAN
ACADV_HUMAN
CLK3_HUMAN
BMX_HUMAN
PRKX_HUMAN
PSME3_HUMAN
SUMO2_HUMAN
RUXE_HUMAN
RUXG_HUMAN
SMD1_HUMAN
SMD2_HUMAN
RSMN_HUMAN
SUMO1_HUMAN
ACTH_HUMAN
GABT_HUMAN
SARNP_HUMAN
OCRL_HUMAN
ROR2_HUMAN
FMR1_HUMAN
PDE4C_HUMAN
CHD3_HUMAN
TADBP_HUMAN
SRSF9_HUMAN
SRSF5_HUMAN
PPR1A_HUMAN
PRP4B_HUMAN
PIN1_HUMAN
ENPP2_HUMAN
PDE7A_HUMAN
CIRBP_HUMAN
NPAT_HUMAN
FRG1_HUMAN
PDE3A_HUMAN
PUM1_HUMAN
NOLC1_HUMAN
SART3_HUMAN
PTPRR_HUMAN
PTPA_HUMAN
KS6A1_HUMAN
SMN_HUMAN
IF16_HUMAN
DUS5_HUMAN
F263_HUMAN
HABP4_HUMAN
PPM1L_HUMAN
USPL1_HUMAN
SPOPL_HUMAN

GEMI2_HUMAN
RU17_HUMAN
FUS_HUMAN
GEMI4_HUMAN
TADBP_HUMAN
SMN_HUMAN
GEMI5_HUMAN
GEMI6_HUMAN
GEMI7_HUMAN
GEMI8_HUMAN
DDX20_HUMAN
STRAP_HUMAN
CBX5_HUMAN
CPSF4_HUMAN
CTCF_HUMAN
LSM11_HUMAN
CPSF1_HUMAN
NPAT_HUMAN
FIP1_HUMAN
SYMPK_HUMAN
LSM10_HUMAN
HINFP_HUMAN
CPSF2_HUMAN
CPSF3_HUMAN
C8AP2_HUMAN
DHX33_HUMAN
WASL_HUMAN
LAT_HUMAN
NPHN_HUMAN
NCK1_HUMAN
AGO2_HUMAN
TNR6B_HUMAN
TAU_HUMAN
POC1A_HUMAN
POC1B_HUMAN
RPOM_HUMAN
MRPP3_HUMAN
PTCD1_HUMAN
MPPB_HUMAN
ERAL1_HUMAN
TRUB2_HUMAN
MAOM_HUMAN
RM19_HUMAN
RT09_HUMAN
GRSF1_HUMAN
FASTK_HUMAN
FAKD1_HUMAN
SYAM_HUMAN
MRM1_HUMAN
RUSD3_HUMAN
SYDM_HUMAN
TM10C_HUMAN
DHX30_HUMAN
FAKD5_HUMAN
NLRX1_HUMAN
SUV3_HUMAN
RM43_HUMAN
NOA1_HUMAN
MCU_HUMAN
PNPT1_HUMAN
PTCD2_HUMAN
TFB1M_HUMAN
RT27_HUMAN
RT31_HUMAN
RUSD4_HUMAN
MTEF3_HUMAN
SUCB2_HUMAN
PEO1_HUMAN
YMEL1_HUMAN
HCD2_HUMAN
MRM3_HUMAN
RM47_HUMAN
DDX28_HUMAN
PAPD1_HUMAN
FAKD2_HUMAN
MRM2_HUMAN
RT07_HUMAN
FMR1_HUMAN
G3BP1_HUMAN
CAPR1_HUMAN
NONO_HUMAN
CPEB4_HUMAN
RBPMS_HUMAN
RTRAF_HUMAN
EWS_HUMAN
RBP56_HUMAN
DDX4_HUMAN

C9JZ20_HUMAN
D6R956_HUMAN
D6R974_HUMAN
D6R991_HUMAN
D6R9P3_HUMAN
D6RAW4_HUMAN
D6RBZ0_HUMAN
D6RD18_HUMAN
D6RE83_HUMAN
D6REM6_HUMAN
D6RHD5_HUMAN
E5RGA2_HUMAN
E5RGR0_HUMAN
E5RHS5_HUMAN
E5RI98_HUMAN
E5RII3_HUMAN
E5RIP5_HUMAN
E5RIU6_HUMAN
E5RJ25_HUMAN
E5RJ48_HUMAN
RS24_HUMAN
E7ESE2_HUMAN
E7ESH4_HUMAN
E7ETD8_HUMAN
E7ETK0_HUMAN
E7EU71_HUMAN
SRBS2_HUMAN
E9PBK7_HUMAN
E9PCW0_HUMAN
E9PD53_HUMAN
E9PEI0_HUMAN
PDC6I_HUMAN
PRDX3_HUMAN
E9PH38_HUMAN
E9PHS0_HUMAN
E9PHT9_HUMAN
E9PI86_HUMAN
E9PIC4_HUMAN
E9PIM9_HUMAN
E9PIP1_HUMAN
E9PJ04_HUMAN
E9PJF0_HUMAN
E9PJT3_HUMAN
E9PJU8_HUMAN
E9PKK4_HUMAN
E9PKR8_HUMAN
E9PKW4_HUMAN
E9PLB3_HUMAN
E9PLC5_HUMAN
E9PLZ3_HUMAN
E9PMD7_HUMAN
E9PMJ3_HUMAN
E9PNS2_HUMAN
E9PNS6_HUMAN
E9PPB5_HUMAN
E9PPJ0_HUMAN
E9PPQ8_HUMAN
E9PQS1_HUMAN
E9PQW4_HUMAN
E9PR95_HUMAN
E9PSC3_HUMAN
F2Z2C0_HUMAN
F5GWA7_HUMAN
F5GWX5_HUMAN
F5GX11_HUMAN
F5GX23_HUMAN
F5GXK7_HUMAN
F5GY37_HUMAN
M2OM_HUMAN
F5GYU3_HUMAN
NONO_HUMAN
F5GZ39_HUMAN
Q5PY61_HUMAN
F5H0C5_HUMAN
F5H169_HUMAN
RL13_HUMAN
F5H265_HUMAN
F5H2D2_HUM

LIPA3_HUMAN
BRE1B_HUMAN
PHF2_HUMAN
CAND2_HUMAN
KDM4A_HUMAN
XYLB_HUMAN
DUS11_HUMAN
PDCD6_HUMAN
TBCA_HUMAN
VATG1_HUMAN
AVIL_HUMAN
H2AY_HUMAN
PEX14_HUMAN
TULP3_HUMAN
CISY_HUMAN
SC22B_HUMAN
DEAF1_HUMAN
MTX2_HUMAN
MPPB_HUMAN
PAGE1_HUMAN
NDUS3_HUMAN
KHDR3_HUMAN
BAF_HUMAN
NPM3_HUMAN
PRDM1_HUMAN
U520_HUMAN
SURF6_HUMAN
PPM1B_HUMAN
UTP20_HUMAN
NU155_HUMAN
TCEA3_HUMAN
POP7_HUMAN
RPP40_HUMAN
DGAT1_HUMAN
DHRS3_HUMAN
SEF2_HUMAN
SPF27_HUMAN
DCTN3_HUMAN
DNJC8_HUMAN
SPF30_HUMAN
DNJB5_HUMAN
UTP25_HUMAN
GLRX3_HUMAN
RL1D1_HUMAN
DFFB_HUMAN
SRP72_HUMAN
DDAH1_HUMAN
RECQ5_HUMAN
AL1A2_HUMAN
ACL6B_HUMAN
KPCD3_HUMAN
NOL4_HUMAN
TOX4_HUMAN
TOX_HUMAN
CDK14_HUMAN
SOGA1_HUMAN
FA13A_HUMAN
RPP14_HUMAN
ALX3_HUMAN
ZN202_HUMAN
CDRT1_HUMAN
VENTX_HUMAN
LC7L3_HUMAN
SPDEF_HUMAN
SNAPN_HUMAN
6PGL_HUMAN
SMC2_HUMAN
AGM1_HUMAN
PARN_HUMAN
TGDS_HUMAN
NSA2_HUMAN
SC24A_HUMAN
ACSL3_HUMAN
RPA1_HUMAN
VATG2_HUMAN
ECEL1_HUMAN
PPR3D_HUMAN
RPP29_HUMAN
LDOC1_HUMAN
HS74L_HUMAN
CLIC3_HUMAN
NUD14_HUMAN
TTLL1_HUMAN
PTTG1_HUMAN
CLPT1_HUMAN
TOM40_H

In [4]:
humanToMouse_id("Q92945")

FUBP2_HUMAN


'Q3U0V1'